### Import Some Neccesary Library

In [1]:
from utils import *
import warnings

warnings.filterwarnings('ignore')

In [2]:
h_theta = np.pi / 180
h_lines = None
h_srn = 0  # divisor for rho
h_stn = 0  # divisor for theta

# Canny
c_t1 = 50
c_t2 = c_t1 * 3

# Process model image=========================
model_img = cv.imread('images/0.jpg')
model_gray = cv.cvtColor(model_img, cv.COLOR_BGR2GRAY)
cv.imwrite("./output/model_gray_scale.jpg", model_gray)
model_canny = cv.Canny(model_gray, c_t1, c_t2)
cv.imwrite("./output/model_canny_detected.jpg", model_gray)
model_lines = cv.cvtColor(model_canny, cv.COLOR_GRAY2BGR)
source_lines = cv.HoughLines(model_canny, 1.1, h_theta, 247, h_lines, h_srn, h_stn)

linesP = cv.HoughLinesP(model_canny, 1, np.pi / 180, 50, None, 50, 10)
cdstP = np.copy(cv.cvtColor(model_canny, cv.COLOR_GRAY2BGR))

if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv.LINE_AA)

cv.imwrite("./output/model_hough_lines.jpg", cdstP)

cartesian_lines = []
if source_lines is not None:
    for i in range(0, len(source_lines)):
        pt1, pt2 = add_line(source_lines, i, model_lines)
        cartesian_lines.append([[pt1[0], pt1[1]], [pt2[0], pt2[1]]])
        
# print("cartesian_lines:", cartesian_lines)
all_intersections = junctions(cartesian_lines)
print("all_intersections:", all_intersections)
cluster_centers = kmeans_centers(all_intersections, 10)

mark_all(cluster_centers, model_lines)
cv.imwrite("model_lines.jpg", model_lines)

# Calculate corners
m_top, m_bot = get_model_corners(cluster_centers)
print("Cluster centers:", cluster_centers)
print("Points on top:", m_top)
print("Points on bot:", m_bot)
model_joined_topbot = m_top + m_bot

all_intersections: [[5, 5], [348, 5], [695, 5], [122, 5], [579, 5], [126, 5], [576, 5], [5, 379], [348, 379], [695, 379], [122, 379], [579, 379], [126, 379], [576, 379], [5, 1], [348, 1], [695, 1], [122, 1], [579, 1], [126, 1], [576, 1]]
Cluster centers: [[124.  379. ]
 [577.5   3. ]
 [124.    3. ]
 [577.5 379. ]
 [348.    3. ]
 [348.  379. ]
 [  5.    3. ]
 [695.    3. ]
 [  5.  379. ]
 [695.  379. ]]
Points on top: [[5, 3], [695, 3]]
Points on bot: [[5, 379], [695, 379]]


In [7]:
image_file = './images/1.jpg'
default_width = 700
frame_raw = cv.imread(image_file)

if frame_raw.shape[1] > default_width:
    frame_raw = image_resizer(image_file, default_width)

src = frame_raw
dst = cv.Canny(src, 50, 200, None, 3)
cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)

cv.imwrite("input-gray-image.jpg", cv.cvtColor(frame_raw, cv.COLOR_BGR2GRAY))
cv.imwrite("input-canny.jpg", cdst)

frame_lines = dst
cartesian_lines = []
lines = cv.HoughLines(dst, 1, np.pi / 180, 110, None, 0, 0)

#Draw hough Lines

if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
        
cv.imwrite("input-image-hough-lines.jpg", cdst)

if lines is not None:
    for i in range(0, len(lines)):
        pt1, pt2 = add_line(lines, i, frame_lines)
        cartesian_lines.append([[pt1[0], pt1[1]], [pt2[0], pt2[1]]])

junction_points = junctions(cartesian_lines)
junction_points_normalized = []

for junction_point in junction_points:
    if (junction_point[0] > 0 and junction_point[1] > 0):
        if (junction_point[0] < src.shape[1] and junction_point[1] > src.shape[0]):
            junction_points_normalized.append(junction_point)

cluster_centers = kmeans_centers(junction_points, 25)
mark_all(junction_points, frame_lines, rbg=(0, 0, 255))

printed = ''
for cluster_center in cluster_centers:
    printed += str(cluster_center) + ', '

print(printed)

warped_img = []
warped_org_image = []
n_noninf = get_noninf(cluster_centers)
if (len(n_noninf) >= 4):
    top, bot = get_frame_corners(n_noninf)
    frame_joined_topbot = top + bot
    if (len(frame_joined_topbot) == 4):
        # Calculate the mask for the warping
        mask, status = cv.findHomography(np.array(frame_joined_topbot), np.array(model_joined_topbot))
        print("Mask:\n", mask)
        # Get frame resolution for cv.warpPerspective()
        height, width, channels = model_img.shape
        warped_img = cv.warpPerspective(frame_raw, mask, (width, height))
        
        inv_h = np.linalg.inv(mask)
        warped_org_image = cv.warpPerspective(warped_img, inv_h, (frame_raw.shape[1], frame_raw.shape[0]), flags=cv.INTER_AREA, borderMode=cv.BORDER_CONSTANT, borderValue=(255, 255, 255))
        
    mark_all(n_noninf, frame_lines)
    print("Clusters: ", len(n_noninf))  # Number of cluster centers on screen

cv.imshow("Model", warped_img)
# cv.imwrite("./output/warped_image.jpg", warped_img)
cv.imwrite("warped_image.jpg", warped_img)
cv.imwrite("warped_org_image.jpg", warped_org_image)
cv.waitKey(0)
cv.destroyAllWindows()

# 6_man
# 14,280 +   [ 26.36708861, 161.91139241]
# 470,463 +  [467.25       407.37068966]
# 620,122 +  [601.65934066 134.13186813]
# 414,49 +  [371.79054054  45.22297297]
# cv.imwrite("warper.jpg", warped_img)

[-42.         424.72727273], [-25988.  -4045.], [-53548. -12114.], [43417.  6145.], [-3309.6  -510. ], [ 589.11363636 -475.70454545], [-2704.18181818  -342.        ], [197.10416667  83.95833333], [-5666.25  -938.5 ], [554.08333333 439.375     ], [-23392.  -4055.], [-2114.75  -226.25], [-26316.  -5820.], [-1149.  -145.], [ 1279. -1468.], [ 61.63636364 270.57575758], [565.52173913 133.69565217], [-4147.5  -510.5], [-3693.4  -563.6], [-2946.16666667  -408.83333333], [-1838.25  -176.25], [ 889.5 -932.5], [367.61904762 112.42857143], [-2426.30769231  -289.38461538], [265.     368.8125], 
D/E:  2.2642677766936488
Points on top: [array([197.10416667,  83.95833333]), array([565.52173913, 133.69565217])]
Points on bot: [array([ 61.63636364, 270.57575758]), array([554.08333333, 439.375     ])]
Mask:
 [[ 3.27231818e+00  2.38882348e+00 -8.38337915e+02]
 [-4.78741060e-01  3.57473615e+00 -2.01440262e+02]
 [ 8.41331027e-04  3.29245278e-03  1.00000000e+00]]
Clusters:  6


QObject::moveToThread: Current thread (0x315a2d0) is not the object's thread (0x2b7dda0).
Cannot move to target thread (0x315a2d0)

QObject::moveToThread: Current thread (0x315a2d0) is not the object's thread (0x2b7dda0).
Cannot move to target thread (0x315a2d0)

Qt: Session management error: Could not open network socket
QObject::moveToThread: Current thread (0x315a2d0) is not the object's thread (0x2b7dda0).
Cannot move to target thread (0x315a2d0)

QObject::moveToThread: Current thread (0x315a2d0) is not the object's thread (0x2b7dda0).
Cannot move to target thread (0x315a2d0)

QObject::moveToThread: Current thread (0x315a2d0) is not the object's thread (0x2b7dda0).
Cannot move to target thread (0x315a2d0)

QObject::moveToThread: Current thread (0x315a2d0) is not the object's thread (0x2b7dda0).
Cannot move to target thread (0x315a2d0)

QObject::moveToThread: Current thread (0x315a2d0) is not the object's thread (0x2b7dda0).
Cannot move to target thread (0x315a2d0)

QObject::moveToT